# Four point functions in the $O(n)$ model

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [4]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools
import BootstrapVirasoro: DoubleGamma
import BootstrapVirasoro.LoopModels: compute_reference
println("Number of threads: ", Threads.nthreads())

  Activating 

Number of threads: 4


project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


## Spectrum

We solve crossing symmetry equations for the spectrum of the $O(n)$ CFT:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(1-x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(1/x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [ ]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c_S2 = CC(β=1 / (big"0.8" + big"0.2" * im))
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
P_S2 = big"0.100000000000000005551115" + big"0.100000000000000005551115"*im
diag_field_S2 = Field(c_S2, P=P_S2)
fields_S2 = vcat([Field(c_S2, r=r, s=s) for (r, s) in ndiag_indices], Field(c_S2, P=P_S2))

# Determine the parity of the number of legs in 4pt channels
function chan_parities(co::Correlation4)
    V1, V2, V3, V4 = co.fields
    chan_parities = Channels{Rational}((V1.r + V2.r) % 1, (V1.r + V4.r) % 1, (V1.r + V3.r) % 1)
end

# Compute series of blocks for any channel field compatible
# with the correlation co
function LoopSpectra(co, fields, fs)
    Vs = @channels filter(V -> V.r % 1 == chan_parities(co)[chan], fields)
    @channels ChannelSpectrum(co, chan, Vs[chan], fs[chan])
end

# Compute series of blocks, keeping only one of each (field, reflected field) pair
# given indices of the external fields.
function precompute_blocks(indices, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2 * precision), base=10)
    Δmax = floor(Int, 1.5 * precision)
    co = Correlation([Field(c_S2, r=r, s=s) for (r, s) in ind], Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    fs = Channels(chan -> (V -> IBlock(co, chan, V, parity=parity)))
    LoopSpectra(co, fields, fs)
end

function compute_diagblocks(specs, Ps, chan)
    c = specs.s.corr.fields[1].c
    return [IBlock(specs.s.corr, chan, Field(c, P=P)) for P in Ps]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
        specs, signature;
        even_spin=Channels(false), 
        rmax=3, show=true, Ps=nothing, diagchan=nothing,
        fix=nothing
    )
    specs = @channels filter(V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan], specs[chan])
    specs = @channels filter(V -> even_spin[chan] ? spin(V) % 2 == 0 : true, specs[chan])
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix)
        if show
            Base.show(stdout, res.str_cst, rmax=rmax)
        end
    else
        diagblocks = compute_diagblocks(specs, Ps, diagchan)
        res = solve_bootstrap_manyP(specs, diagblocks)
    end
    return res
end

solve (generic function with 1 method)

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [35]:
ind = ((1//2, 0), (1//2, 0), (1//2, 0), (1//2, 0))
blocks = precompute_blocks(ind, fields_S2, parity=0, precision=10);

### Signature $(0, 1, 1)$

In [12]:
solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false), rmax=2)

 Channel s
┌─────────────────┬───────────────────────────┬────────────────┐
│ Field           │ Structure constant        │ Relative error │
├─────────────────┼───────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                 1.0+0.0im │            0.0 │
│          (1, 0) │      -0.209496+0.113616im │     3.06327e-8 │
│          (2, 0) │ -0.00301279+0.000871235im │     4.19914e-7 │
│          (2, 1) │  0.00101903-0.000847696im │     3.93445e-7 │
└─────────────────┴───────────────────────────┴────────────────┘
 Channel t
┌────────────┬──────────────────────────┬────────────────┐
│ Field      │ Structure constant       │ Relative error │
├────────────┼──────────────────────────┼────────────────┤
│     (1, 0) │        0.5221+0.127106im │     1.45609e-8 │
│     (1, 1) │      0.20788-0.0478817im │     5.12643e-9 │
│     (2, 0) │ -0.0011054+0.000890287im │     7.99613e-6 │
│ (2, -1//2) │ -0.00107558+0.00116214im │     8.64958e-6 │
│  (2, 1//2) │ -0.00107558+0.00116216im │    

BootstrapSystem{Complex{BigFloat}}
< (1//2, 0) (1//2, 0) (1//2, 0) (1//2, 0) >
Number of positions: 28
Matrix size: (56, 50)


### Signature $(0, 1, 0)$, $D^{(s)}_{(2, 0)} = 0$

In [11]:
fix = [(:s, diag_field_S2, 1), (:s, Field(c_S2, r=2, s=0), 0)]
solve(blocks, Sig(0, 1, 0), fix=fix, rmax=2)

 Channel s
┌─────────────────┬──────────────────────────┬────────────────┐
│ Field           │ Structure constant       │ Relative error │
├─────────────────┼──────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                1.0+0.0im │            0.0 │
│          (1, 0) │      -1.16516-0.591059im │     3.16594e-5 │
│          (1, 1) │     0.464568+0.0798304im │     3.48287e-5 │
│          (2, 0) │                0.0+0.0im │            0.0 │
│      (2, -1//2) │ -0.00318408+0.00145069im │     2.98446e-5 │
│       (2, 1//2) │ -0.00318421+0.00145079im │     5.53319e-5 │
│          (2, 1) │  0.00257343-0.00274975im │     3.66843e-5 │
└─────────────────┴──────────────────────────┴────────────────┘
 Channel t
┌────────────┬──────────────────────────┬────────────────┐
│ Field      │ Structure constant       │ Relative error │
├────────────┼──────────────────────────┼────────────────┤
│     (1, 0) │     -0.433562-0.577569im │     5.72741e-5 │
│     (1, 1) │     0.672448+0.0319487i

BootstrapSystem{Complex{BigFloat}}
< (1//2, 0) (1//2, 0) (1//2, 0) (1//2, 0) >
Number of positions: 34
Matrix size: (68, 60)


### Solve for many different Ps

In [36]:
# solve for many different values of the momentum for the diag field in the channel.
# only recompute one column of the linear system each time: very cheap.
Ps = [big"0.8" + big"0.2"*im + i / big(100)*im for i in 1:3];
diagblocks = compute_diagblocks(blocks, Ps, :s);;
sol_Ps = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),  Ps=Ps, diagchan=:s);
show(stdout, sol_Ps[2].str_cst, rmax=2)

 Channel s
┌─────────────────┬──────────────────────────┬────────────────┐
│ Field           │ Structure constant       │ Relative error │
├─────────────────┼──────────────────────────┼────────────────┤
│          (1, 0) │      0.253897-0.354829im │     1.09937e-7 │
│ (P=0.80+0.22im) │                1.0+0.0im │            0.0 │
│          (2, 0) │  0.00488132-0.00418449im │     7.47749e-6 │
│          (2, 1) │ -0.00118272+0.00244771im │     3.18973e-6 │
└─────────────────┴──────────────────────────┴────────────────┘
 Channel t
┌────────────┬─────────────────────────┬────────────────┐
│ Field      │ Structure constant      │ Relative error │
├────────────┼─────────────────────────┼────────────────┤
│     (1, 0) │     -1.08261+0.205966im │     8.00592e-9 │
│     (1, 1) │    -0.347437+0.265879im │     4.60317e-8 │
│     (2, 0) │  0.0047108-0.00153185im │     4.43764e-6 │
│ (2, -1//2) │ 0.00384689-0.00253432im │     6.96987e-7 │
│  (2, 1//2) │ 0.00384687-0.00253435im │     5.29287e-6 │
│ 

Benchmarks: on my 2015, 4-core Intel Macbook pro.
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

In [18]:
ss = rand(Complex{BigFloat}, 4)
ind = Tuple((0, s) for s in ss)
blocks = precompute_blocks(ind, fields_S2, parity=1, precision=15);

In [19]:
sol = solve(blocks, Sig(0, 0, 0), show=true);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │      -0.0832456+0.178134im │    4.91227e-12 │
│          (1, 1) │      0.0201009+0.0315986im │    7.41516e-12 │
│          (2, 0) │   0.000240425-0.00019737im │     4.69748e-9 │
│       (2, 1//2) │ -1.81616e-06-8.39333e-05im │     1.09173e-8 │
│          (2, 1) │ -0.000120737-0.000115819im │     2.11426e-9 │
│          (3, 0) │ -1.60164e-08-1.46858e-08im │     0.00158422 │
│       (3, 1//3) │ -4.62642e-09+4.56308e-09im │     0.00481834 │
│       (3, 2//3) │ -1.51129e-08+9.00136e-09im │     0.00137446 │
│          (3, 1) │ -4.27944e-09+5.89639e-10im │     0.00360355 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────────────┬──────

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [21]:
ind = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))
blocks = precompute_blocks(ind, fields_S2, parity=1, precision=13);

### Signature $(0, \frac32, \frac32)$

In [22]:
sol = solve(blocks, Sig(0, 3//2, 3//2));

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │       -0.258114+0.242755im │    3.96531e-10 │
│          (1, 1) │                          0 │       0.939255 │
│          (2, 0) │  0.000807871+7.24101e-05im │     2.23197e-7 │
│       (2, 1//2) │                          0 │        0.61138 │
│          (2, 1) │                          0 │       0.870137 │
│          (3, 0) │   8.1891e-06-4.28143e-06im │     9.49991e-6 │
│       (3, 1//3) │                          0 │       0.865817 │
│       (3, 2//3) │ -4.76461e-06+3.96243e-06im │     1.18585e-5 │
│          (3, 1) │                          0 │       0.869367 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌──────────────┬────────────────────────────┬─────────

### Signature $(1, \frac12, \frac32)$

In [25]:
solve(blocks, Sig(1, 1//2, 3//2), rmax=2);

 Channel s
┌───────────┬─────────────────────────┬────────────────┐
│ Field     │ Structure constant      │ Relative error │
├───────────┼─────────────────────────┼────────────────┤
│    (1, 0) │               1.0+0.0im │            0.0 │
│    (1, 1) │     0.485267-0.172024im │    1.17811e-10 │
│    (2, 0) │  -0.0095726+0.0442801im │     3.53547e-9 │
│ (2, 1//2) │ -0.00338609+0.0250186im │    7.80266e-10 │
│    (2, 1) │                       0 │        0.80249 │
└───────────┴─────────────────────────┴────────────────┘
 Channel t
┌──────────────┬────────────────────────┬────────────────┐
│ Field        │ Structure constant     │ Relative error │
├──────────────┼────────────────────────┼────────────────┤
│    (1//2, 0) │     1.28685+0.161072im │    1.26397e-10 │
│    (3//2, 0) │  -0.162512+0.0803764im │    3.52806e-10 │
│ (3//2, 2//3) │ -0.0198012+0.0279922im │    9.31013e-10 │
└──────────────┴────────────────────────┴────────────────┘
 Channel u
┌──────────────┬──────────────────────┬──

### Signature $(1, \frac32, \frac12)$

In [26]:
sol = solve(blocks, Sig(1, 3//2, 1//2), rmax=2);

 Channel s
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (1, 0) │              1.0+0.0im │            0.0 │
│    (1, 1) │   -0.485267+0.172024im │    1.25958e-10 │
│    (2, 0) │ -0.0095726+0.0442801im │     4.01997e-9 │
│ (2, 1//2) │ 0.00338609-0.0250186im │     1.38921e-9 │
│    (2, 1) │                      0 │       0.531479 │
└───────────┴────────────────────────┴────────────────┘
 Channel t
┌──────────────┬───────────────────────┬────────────────┐
│ Field        │ Structure constant    │ Relative error │
├──────────────┼───────────────────────┼────────────────┤
│    (3//2, 0) │   0.873083+0.274474im │    6.18514e-11 │
│ (3//2, 2//3) │ -0.280453-0.0181805im │    1.87174e-10 │
└──────────────┴───────────────────────┴────────────────┘
 Channel u
┌──────────────┬───────────────────────┬────────────────┐
│ Field        │ Structure constant    │ Relative error │

## $\left\langle (0, \frac12)^3 P \right\rangle$

In [28]:
ind = ((0, 1//2), (0, big"0.2"+big"0.4"*im), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields_S2, parity=1, precision=20);
solve(blocks, Sig(0, 0, 0), even_spin=Channels(true), rmax=3);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │       0.0158377+0.123451im │     3.2272e-12 │
│          (2, 0) │ -0.000252742-0.000371888im │     7.2221e-10 │
│          (2, 1) │ -0.000102699+6.14249e-05im │    5.00903e-10 │
│          (3, 0) │  -2.41914e-08+2.7827e-08im │     2.37535e-5 │
│       (3, 2//3) │  1.45169e-08+2.70988e-08im │     1.00931e-5 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │          1.0+2.79532e-13im │     1.3573e-13 │
│          (1, 0) │       0.0158377+0.123451im │    3.

## $\left\langle (0, \frac12)^3 (1, 0) \right\rangle$

In [29]:
ind = ((1, 0), (0, 1//2), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields_S2, parity=1, precision=20);

In [30]:
sys_S2 = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false), rmax=2);

 Channel s
┌─────────────────┬───────────────────────────┬────────────────┐
│ Field           │ Structure constant        │ Relative error │
├─────────────────┼───────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                 1.0+0.0im │            0.0 │
│          (1, 0) │                         0 │        1.08685 │
│          (2, 0) │ -0.00585115-0.000436412im │    6.71336e-11 │
│          (2, 1) │  0.00227685-0.000981606im │    6.13855e-11 │
└─────────────────┴───────────────────────────┴────────────────┘
 Channel t
┌───────────┬──────────────────────────┬────────────────┐
│ Field     │ Structure constant       │ Relative error │
├───────────┼──────────────────────────┼────────────────┤
│    (1, 0) │      0.676571+0.310673im │    1.07024e-12 │
│    (1, 1) │     0.286591-0.0446828im │    6.33833e-13 │
│    (2, 0) │ -0.00437577+0.00233119im │     2.6952e-10 │
│ (2, 1//2) │ -0.00327768+0.00228679im │     2.6645e-10 │
│    (2, 1) │ -0.00118392+0.00172743im │    2.17233e

## $\left\langle (0, \frac12)^3 (2, 0) \right\rangle$

In [32]:
ind = ((0, 1//2), (2, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields_S2, parity=1, precision=20);
solve(blocks, Sig(1, 2, 1), even_spin=Channels(false, true, false), rmax=3);

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │        -0.63854+0.181026im │    2.25391e-13 │
│    (2, 0) │                          0 │       0.655262 │
│ (2, 1//2) │                          0 │        1.74904 │
│    (2, 1) │                          0 │        0.72868 │
│    (3, 0) │ -7.86522e-05-0.000322813im │     7.24588e-8 │
│ (3, 1//3) │   4.37238e-05+0.00014278im │     1.51095e-7 │
│ (3, 2//3) │  4.92031e-05+9.64753e-05im │      1.4437e-7 │
│    (3, 1) │ -8.77471e-05-9.14424e-05im │     7.73184e-8 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (2, 0) │    0.413186+0.31

## $\left\langle (0, \frac12)^3 (3, 0) \right\rangle$

In [33]:
ind = ((0, 1//2), (3, 0), (0, 1//2), (0, 1//2))
blocks_even_S2 = precompute_blocks(ind, fields_S2, parity=1, precision=20);
solve(blocks_even_S2, Sig(2, 2, 2), even_spin=Channels(true), rmax=3);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (2, 0) │            1.0+0.0im │            0.0 │
│    (2, 1) │ -0.452015+0.191135im │    1.86263e-12 │
│    (3, 0) │                    0 │        6.54526 │
│ (3, 2//3) │                    0 │        2.68981 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (2, 0) │    1.0-8.92538e-15im │    5.76547e-13 │
│    (2, 1) │ -0.452015+0.191135im │    5.69053e-13 │
│    (3, 0) │                    0 │       0.984152 │
│ (3, 2//3) │                    0 │        3.59343 │
└───────────┴──────────────────────┴────────────────┘
 Channel u
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative err